In [1]:
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical
from hyperas import optim
from hyperas.distributions import choice, conditional, uniform
from hyperopt import Trials, STATUS_OK, tpe

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Load the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((-1, 28 * 28))
X_train = X_train.astype("float32") / 255
X_test = X_test.reshape((-1, 28 * 28))
X_test = X_test.astype("float32") / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 784), (10000, 784), (60000, 10), (10000, 10))

In [6]:
def data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape((-1, 28 * 28))
    X_train = X_train.astype("float32") / 255
    X_test = X_test.reshape((-1, 28 * 28))
    X_test = X_test.astype("float32") / 255

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    return X_train, y_train, X_test, y_test

def create_model(X_train, y_train, X_test, y_test):
    model = models.Sequential()
    model.add(layers.Dense(512, activation="relu", input_shape=(784,)))
    model.add(layers.Dropout({{uniform(0, 1)}}))
    model.add(layers.Dense({{choice([256, 512])}}, activation="relu"))
    model.add(layers.Dropout({{uniform(0, 1)}}))
    model.add(layers.Dense(10, activation="softmax"))
    
    model.compile(optimizer={{choice(["rmsprop", "adam", "sgd"])}},
                  loss="categorical_crossentropy",
                  metrics=["acc"])
    
    model.fit(X_train, y_train,
              epochs=1,
              batch_size={{choice([32, 64, 128])}},
              validation_split=0.2, verbose=2)
    
    score, acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy is: {acc * 100:.2f}%")
    
    return {"loss": -acc, "status": STATUS_OK, "model": model}

In [7]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(), notebook_name="Untitled1")

>>> Imports:
#coding=utf-8

try:
    from keras import layers, models
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, conditional, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': hp.choice('Dense', [256, 512]),
        'Dropout_1': hp.uniform('Dropout_1', 0, 1),
        'optimizer': hp.choice('optimizer', ["rmsprop", "adam", "sgd"]),
        'batch_size': hp.choice('batch_size', [32, 64, 128]),
    }

>>> Data
  1: 
  2: (X_train, y_train), (X_test, y_test) = mnist.load_data()
  3: X_train = X_train.reshape((-1, 28 * 28))
  4: X_train = X_train.astype("float32") / 255
  5: X_test = X_test.reshape((-1, 28 * 28))
  6: X_tes

In [8]:
best_run

{'Dense': 1,
 'Dropout': 0.587606728324542,
 'Dropout_1': 0.3746350041674067,
 'batch_size': 2,
 'optimizer': 0}